**Importiere die notwendigen Packages**

In [1]:
# Empfehlung für Colab: Installiere alles was Du brauchst
# %pip install pandas pivottablejs numpy altair matplotlib geopandas folium

In [2]:
import pandas as pd
import pivottablejs
from pivottablejs import pivot_ui
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
from datetime import datetime
import geopandas as gpd
import folium 



Definiere Settings. Hier das Zahlenformat von Float-Werten (z.B. *'{:,.2f}'.format* mit Komma als Tausenderzeichen), 

In [3]:
pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 15)

Importiere einen Datensatz von der **DWH_INT** Dropzone.

- Beachte dabei die Notation des Pfades...
- Definiere mal aktuell noch keine weiteren Parameter beim Import

**Dataset auf INTEG Datenkatalog**: https://data.integ.stadt-zuerich.ch/dataset/int_dwh_bau_whg_bestand_umbau_ea_zizahlkat_kreis_saldo_jahr_od5101

(int_dwh)

In [4]:
#Die Datasets sind nur zum Testen auf INT-DWH-Dropzone. Wenn der Test vorbei ist, sind sie auf PROD. 
# Über den Status kann man einfach switchen

status = "int";
print(status)

int


In [5]:
# Filepath
if status == "prod":
    fp = r"\\szh\ssz\applikationen\OGD_Dropzone\DWH\bau_whg_bestand_umbau_ea_zizahlkat_kreis_saldo_jahr_OD5101\BAU510OD5101.csv"
else:
    fp = r"\\szh\ssz\applikationen\OGD_Dropzone\INT_DWH\bau_whg_bestand_umbau_ea_zizahlkat_kreis_saldo_jahr_OD5101\BAU510OD5101.csv"
print(fp)

\\szh\ssz\applikationen\OGD_Dropzone\INT_DWH\bau_whg_bestand_umbau_ea_zizahlkat_kreis_saldo_jahr_OD5101\BAU510OD5101.csv


In [6]:
# Read the data
data2betested = pd.read_csv(
    fp, 
    sep=',',
   #usecols=[0,1,2,3,4,6],
   #names=['stationid','year','month','avgtemp','latitude','longitude','elevation','station','countryid','country'],
   #skiprows=1,
   # parse_dates=[0],
    parse_dates=True,
    #date_parser=lambda x: pd.to_datetime(x, format="%Y"), 
    #index_col="date_time",
    low_memory=False    
    )
print('done')

done


In der Folge ein paar erste Tests:

In [7]:
type(data2betested)

pandas.core.frame.DataFrame

 - 1) Zeige eine kurze Vorschau der importierten Daten
 - 2) Weise die Datentypen aus
 - 3) Zeige die Shape (Umfang) des Datensatzes an

In [10]:
#data2betested.head(3)

In [11]:
data2betested.dtypes

Jahr                 int64
KreisCd              int64
KreisLang           object
EigentumsartCd       int64
EigentumsartLang    object
Wohnungsgroesse     object
Wohnungssaldo        int64
dtype: object

In [12]:
data2betested.shape

(486, 7)

Beschreibe einzelne Attribute

In [13]:
data2betested.describe()

,Jahr,KreisCd,EigentumsartCd,Wohnungssaldo
count,486,486,486,486
mean,2018,7,3,4
std,2,3,1,14
min,2015,1,1,-69
25%,2016,3,3,0
50%,2018,7,3,1
75%,2020,10,4,5
max,2021,12,5,92


Wie viele Nullwerte gibt es im Datensatz?

In [14]:
data2betested.isnull().sum()

Jahr                0
KreisCd             0
KreisLang           0
EigentumsartCd      0
EigentumsartLang    0
Wohnungsgroesse     0
Wohnungssaldo       0
dtype: int64

Wandle die Spalte Jahr von Int zu **Datetime** um. Dies wird in den meisten Grafiken so benötigt.

In [15]:
data2betested['Jahr']=pd.to_datetime(data2betested['Jahr'], format='%Y')
#data2betested['Jahr']=pd.to_datetime(data2betested['Jahr'], format='%Y')
#TO DO --> setze das Datum auf den 31.12. der Jahres.
#test=data2betested[['StichtagDatJahr']].date().replace(month=12, day=31)

data2betested


,Jahr,KreisCd,KreisLang,EigentumsartCd,EigentumsartLang,Wohnungsgroesse,Wohnungssaldo
0,2015-01-01,1,Kreis 1,3,Übrige private Gesellschaften,AnzGbdWhgStatZim1_3_saldo,12
1,2015-01-01,1,Kreis 1,3,Übrige private Gesellschaften,AnzGbdWhgStatZim4plus_saldo,0
2,2015-01-01,1,Kreis 1,4,Natürliche Personen,AnzGbdWhgStatZim1_3_saldo,3
3,2015-01-01,1,Kreis 1,4,Natürliche Personen,AnzGbdWhgStatZim4plus_saldo,2
4,2015-01-01,10,Kreis 10,3,Übrige private Gesellschaften,AnzGbdWhgStatZim1_3_saldo,0
...,...,...,...,...,...,...,...
481,2021-01-01,9,Kreis 9,3,Übrige private Gesellschaften,AnzGbdWhgStatZim4plus_saldo,0
482,2021-01-01,9,Kreis 9,4,Natürliche Personen,AnzGbdWhgStatZim1_3_saldo,-4
483,2021-01-01,9,Kreis 9,4,Natürliche Personen,AnzGbdWhgStatZim4plus_saldo,13
484,2021-01-01,9,Kreis 9,5,Im Stockwerkeigentum,AnzGbdWhgStatZim1_3_saldo,3


Bereite die Daten für die Grafik unten vor

In [16]:
Jahre = data2betested.groupby("Jahr")
len(Jahre)

7

In [17]:
for key, group in Jahre:
    print(key, len(group))

2015-01-01 00:00:00 56
2016-01-01 00:00:00 74
2017-01-01 00:00:00 70
2018-01-01 00:00:00 74
2019-01-01 00:00:00 68
2020-01-01 00:00:00 72
2021-01-01 00:00:00 72


In [20]:
myJahr = "2021-01-01"
aktuellesJahr=Jahre.get_group(myJahr)   
aktuellesJahr.head(3)


,Jahr,KreisCd,KreisLang,EigentumsartCd,EigentumsartLang,Wohnungsgroesse,Wohnungssaldo
414,2021-01-01,1,Kreis 1,3,Übrige private Gesellschaften,AnzGbdWhgStatZim1_3_saldo,8
415,2021-01-01,1,Kreis 1,3,Übrige private Gesellschaften,AnzGbdWhgStatZim4plus_saldo,1
416,2021-01-01,1,Kreis 1,4,Natürliche Personen,AnzGbdWhgStatZim1_3_saldo,0


**Selektionen und Aggregationen**

Check this [Doku](https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/)

In [21]:
   
test = data2betested.groupby(
   ['Jahr', 'Wohnungsgroesse']
).agg(
    {
         'Wohnungssaldo':(sum),     # Sum --> (sum)
         #'AnzPers': (sum),   
         #'Wohnungsfl': 'max'  
    }
)
test.head(10)


Wohnungssaldo
Jahr       Wohnungsgroesse                           
2015-01-01 AnzGbdWhgStatZim1_3_saldo              202
           AnzGbdWhgStatZim4plus_saldo             28
2016-01-01 AnzGbdWhgStatZim1_3_saldo              253
           AnzGbdWhgStatZim4plus_saldo             57
2017-01-01 AnzGbdWhgStatZim1_3_saldo              303
           AnzGbdWhgStatZim4plus_saldo             46
2018-01-01 AnzGbdWhgStatZim1_3_saldo              269
           AnzGbdWhgStatZim4plus_saldo             -5
2019-01-01 AnzGbdWhgStatZim1_3_saldo              155
           AnzGbdWhgStatZim4plus_saldo             15

**Einfache Visualisierungen zur Plausi**

Exploriere die Daten mit Pivottable.JS

In [22]:
from pivottablejs import pivot_ui

pivot_ui(data2betested)

Liniendiagramm 
[Link zur Doku](https://altair-viz.github.io/gallery/multiline_highlight.html)

In [23]:
mySelection = aktuellesJahr[['Jahr', 'KreisCd', 'KreisLang', 'EigentumsartCd', 'EigentumsartLang', 'Wohnungssaldo']]
mySelection

,Jahr,KreisCd,KreisLang,EigentumsartCd,EigentumsartLang,Wohnungssaldo
414,2021-01-01,1,Kreis 1,3,Übrige private Gesellschaften,8
415,2021-01-01,1,Kreis 1,3,Übrige private Gesellschaften,1
416,2021-01-01,1,Kreis 1,4,Natürliche Personen,0
417,2021-01-01,1,Kreis 1,4,Natürliche Personen,0
418,2021-01-01,10,Kreis 10,1,Öffentliche Hand,0
...,...,...,...,...,...,...
481,2021-01-01,9,Kreis 9,3,Übrige private Gesellschaften,0
482,2021-01-01,9,Kreis 9,4,Natürliche Personen,-4
483,2021-01-01,9,Kreis 9,4,Natürliche Personen,13
484,2021-01-01,9,Kreis 9,5,Im Stockwerkeigentum,3


In [24]:
agg_mySelection = mySelection.groupby(
   ['Jahr', 'KreisCd', 'KreisLang', 'EigentumsartCd', 'EigentumsartLang']
).agg(
    {
         'Wohnungssaldo':(sum,min, max)     #, 'AnzPers': (sum, min, max)
    }
)
agg_mySelection.head()

Wohnungssaldo        
                                                                                    sum min max
Jahr       KreisCd KreisLang EigentumsartCd EigentumsartLang                                   
2021-01-01 1       Kreis 1   3              Übrige private Gesellschaften             9   1   8
                             4              Natürliche Personen                       0   0   0
           2       Kreis 2   3              Übrige private Gesellschaften             9   3   6
                             4              Natürliche Personen                       1   0   1
           3       Kreis 3   3              Übrige private Gesellschaften            -1  -4   3

In [25]:
mySelection = aktuellesJahr[(aktuellesJahr.EigentumsartCd == 2) | (aktuellesJahr.EigentumsartCd == 4)]

mySelection.head()

,Jahr,KreisCd,KreisLang,EigentumsartCd,EigentumsartLang,Wohnungsgroesse,Wohnungssaldo
416,2021-01-01,1,Kreis 1,4,Natürliche Personen,AnzGbdWhgStatZim1_3_saldo,0
417,2021-01-01,1,Kreis 1,4,Natürliche Personen,AnzGbdWhgStatZim4plus_saldo,0
420,2021-01-01,10,Kreis 10,2,Wohnbaugenossenschaften,AnzGbdWhgStatZim1_3_saldo,-5
421,2021-01-01,10,Kreis 10,2,Wohnbaugenossenschaften,AnzGbdWhgStatZim4plus_saldo,0
424,2021-01-01,10,Kreis 10,4,Natürliche Personen,AnzGbdWhgStatZim1_3_saldo,-1


In [26]:
# df_statzonen = df_statzonen[(df_statzonen.knr>0) & (df_statzonen.qname!="Hirslanden")]

highlight = alt.selection(type='single', on='mouseover',
                          fields=['Wohnungsgroesse'], nearest=True)
#x='date:StichtagDatJahr',
base = alt.Chart(mySelection).encode(
    x='KreisLang',
    y='Wohnungssaldo',
    color='Wohnungsgroesse'
)

points = base.mark_circle().encode(
    opacity=alt.value(0)
).add_selection(
    highlight
).properties(
    width=700 , height=300
)
lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(4))
).interactive()

points + lines

alt.LayerChart(...)

In [27]:
alt.Chart(mySelection).mark_circle(size=60).encode(
    x='Wohnungssaldo',
    y='KreisLang',
    #color='AnzWhg',
    #color=alt.Color('EigentumsartLang', scale=alt.Scale(scheme='dark2')),
    color=alt.Color('Wohnungssaldo', scale=alt.Scale(scheme='greens')),
    tooltip=['Jahr', 'KreisCd', 'KreisLang', 'EigentumsartCd', 'EigentumsartLang', 'Wohnungsgroesse']
).interactive() # this makes the axes interactive: now you can zoom & pan




alt.Chart(...)